In [1]:
# Some useful settings for interactive work
%load_ext autoreload
%autoreload 2

%matplotlib widget

In [2]:
# Import the relevant modules
import sousvide.synthesize.rollout_generator as rg
import sousvide.synthesize.observation_generator as og
import sousvide.instruct.train_policy as tp
import sousvide.visualize.plot_synthesize as ps
import sousvide.visualize.plot_learning as pl
import sousvide.flight.deploy_figs as df

/home/admin/anaconda3/envs/sv-env/lib/python3.10/site-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: '2.0.1' (you have '2.0.0'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [3]:
# FiGS examples (scene_name, course_name)
flights = [
    ('mid_gate','robustness_track'),
    ]

roster = [
    "Viper",        # HP-Net (non-visual policy)
    "Maverick",     # SV-Net
    "Iceman",       # SV-Net with Direct History (no latent encoding)
    "Hollywood",    # SV-Net with no Pre-Training
    "Cougar"        # SV-Net with no RMA
    ]

drone = "carl"
method = "beta"
cohort = "robustness"

Nep_par = 50
Nep_com = 200

In [ ]:
# Generate Rollouts
rg.generate_rollout_data(cohort,method,flights)

ps.plot_samples(cohort)

Cohort : robustness
Method : alpha
Policy : vrmpc_fr
Frame  : carl
Flights: [('mid_gate', 'robustness_track')]
Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


Loading latest checkpoint from load_dir

✅ Done loading checkpoint from outputs/mid_gate/splatfacto/2024-10-07_145741/nerfstudio_models/step-000029999.ckpt

--------------------------------------------------------------------------
Course Name : robustness_track
Rollout Reps: 2 (per time point)
Rollout Rate: 1 (per second)
Rollout Data: 40 (per sample)
--------------------------------------------------------------------------
Total Rollouts: 30
Batch Sizes : 0 x 50 + 1 x 30
Total Data: 1200
--------------------------------------------------------------------------


  0%|          | 0/1 [00:00<?, ?it/s]

/home/admin/anaconda3/envs/sv-env/lib/python3.10/site-packages/torch/_inductor/compile_fx.py:135: UserWarning: TensorFloat32 tensor cores for float32 matrix multiplication available but not enabled. Consider setting `torch.set_float32_matmul_precision('high')` for better performance.
  warnings.warn(


In [ ]:
# Generate Observations for Datya Size Variation
og.generate_observation_data(cohort,roster)

In [ ]:
# Train the History Network
tp.train_roster(cohort,roster,"Parameter",Nep_par,lim_sv=10)

# Review Losses
pl.plot_losses(cohort,roster,"Parameter")

In [ ]:
# Train the Command Network
tp.train_roster(cohort,roster,"Commander",Nep_com,lim_sv=10)

# Review Losses
pl.plot_losses(cohort,roster,"Commander")

In [ ]:
# Simulate in FiGS
for scene,course in flights:
    df.simulate_roster(cohort,method,scene,course,roster)

# Plot the results
for _,course in flights:
    pl.review_simulations(cohort,course,roster,plot_show=True)